In [1]:
#import libraries
import requests
import pandas as pd
import time

In [2]:
#Keys
API_KEY = "" #Enter your API Key here
CHANNEL_ID = "" #Enter Channel ID for Channel you want to grab information from

In [3]:
def get_video_details(video_id):

    #collecting view, like,comment counts(as of the creation of this project, Youtube has removed the ability to see dislike count on Youtube videos)
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    ###dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count,comment_count

In [4]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) 
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count,comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            #Check if there is a next page, if not, break out 
            if response['nextPageToken'] != None: 
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [5]:
#main

#build our dataframe
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"]) 

df = get_videos(df)

In [7]:
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,ZNCb-BFe7Xk,I tried to make an award show... I got roasted...,2022-01-04,534303,28687,632
1,HLbUUBl1NPU,React Streamers have gone too far...,2022-01-02,673518,36545,1767
2,vVyhrtuz0r4,This VTuber Challenged me... she didn&#39;t re...,2022-01-01,726240,44505,1403
3,viqLJiJ3-uk,the funniest tik toks of 2021,2021-12-31,641093,34727,623
4,HspbLVenr7Q,Can you peel a raw egg?,2021-12-30,719606,50132,1570
...,...,...,...,...,...,...
496,xKTJxzgdi2w,LAST TO LAUGH WINS $2000,2019-11-13,1308137,35126,524
497,YObMPFwhEPg,MY SECRET BEST FRIEND,2019-11-10,135396,3309,119
498,riGjrYWCvZ4,How I became the fastest masher in the world,2019-11-08,1801809,54338,1492
499,du70TojOi_A,YOU CANNOT LAUGH,2019-10-24,2960572,66097,1030


In [8]:
df.to_csv('youtube_vids.csv')